In [9]:
# Problem 1: Build a simple, list based k-mer index of a string to be searched

#Example adapted from Ben Langmead (thanks!)
#Complete!

import bisect
import sys

class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        
        self.t = t
        
        #considerations for later:
        self.k = k  # k-mer length (k)
        
        self.index = []
        
        #creates a substring index w/parameters t = text and k = substring size 
        for offset in range(0,len(self.t)-self.k+1):
            #enumerating kmers
            kmer = self.t[offset: offset+self.k]
            #saving them and their offsets as tuples
            self.index.append((kmer, offset))
            
        #sorting the substring index attribute
        self.index.sort()
        
       
        
    #complete
    def queryKmer(self, kmer):
        ''' Return locations of kmer in t'''
        
        #take sorted substring index, kmer and start binary search
        
        assert len(kmer) == self.k
        hits = [] 
        
      # Find first location of kmer in self.index (hint: use bisect.bisect_left function)
    
        #first needs to run through an alphabetized list of the substrings and find the first hit
        justSubStrings = [tuple[0] for tuple in self.index]
        firstHit = bisect.bisect_left(justSubStrings, kmer)
        
        #using the firstHit index (which gives the index in self.index) to find the first occurence in the text
        firstOccurence = self.index[firstHit][1]
        
        
      #Iterate through self.index from first location of kmer to last adding matches to hits
        for tuple in self.index[firstOccurence:]:
            if tuple[0] == kmer:
                #add the occurences to hits
                hits.append(tuple[1])   
        
        return hits
    
    def query(self, p):
        ''' Return occurrences of pattern p in t'''
        kmer = p[:self.k]
        
        occurrences = []
        
      # Code to complete:
      # Use self.queryKmer to find locations of prefix kmer of p in t
        kmerHits = self.queryKmer(kmer)
        
      # For each location, ascertain if suffix of p matches the corresponding substring of t
        
        #iterate through each hit
        for hit in kmerHits:
            
            #start character comparing from the end of the kmer
            
            #iterate over all of the characters of the suffix after the hit
            for index in range(0,len(p)-self.k+1):
                textCharacter = self.t[hit+index:hit+index+1]
                #get the proper pattern character in the corresponding index
                patternCharacter = p[index:index+1]
                
                #Boolean should only remain true if the entire suffix matches the remainder of the pattern
                found = True
                   
                if patternCharacter != textCharacter:
                        found = False
                        break
            #if found stays true, append the hit to the occurences  
            if found:
                occurrences.append(hit)

      # Return the occurrences
        
        return occurrences
       
text = 'ACTTGGAGATCTTTGAGGCTAGGTATTCGGGATCGAAGCTCATTTCGGGGATCGATTACGATATGGTGGGTATTCGGGA'
pattern = 'GGTATTCGGGA'
K = 3

index = Index(text, K)
firstHit = index.queryKmer(kmer="GGT")
patternOccurrences = index.query(pattern)

In [10]:
# Test queryKmer method
text = 'ACTTGGAGATCTTTGAGGCTAGGTATTCGGGATCGAAGCTCATTTCGGGGATCGATTACGATATGGTGGGTATTCGGGA'
pattern = 'GGTATTCGGGA'
K = 3

index = Index(text, K)
index.queryKmer("GGT") == [21, 64, 68]

True

In [11]:
# Test query method
index.query(pattern) == [21, 68]

True

In [ ]:
# Report index specificity 
#what is this??
float(len(index.query(pattern)))/len(index.queryKmer(pattern[:K]))

In [4]:
# Problem 2: Build a simple suffix array

#complete
class SuffixArray(object):
    #complete
    def __init__(self, t):
        ''' Create suffix array representing suffixes in t '''
      
        self.td = t + "$"
        self.index = [] ## Array of integers representing lexicographically sorted suffixes of t
        # e.g. for t$ = ATA$
        # have suffixes
        # 0 = ATA$
        # 1 = TA$
        # 2 = A$
        # 3 = $
        # such that self.index == [ 3, 2, 0, 1 ]
      
    
        #iterate through all of the start indices for each suffix and sort in lexographic order
        #lambda function fetches the actual suffix from each start index and uses that as its key
        self.index = sorted([end for end in range(0,len(self.td))], key = lambda a: self.td[a:])
      
    
    def query(self, p):
        # function on self.index
        ''' Return occurrences of pattern p in t'''
      
        # Code to complete - find all occurrences of p in t by writing binary search
        
        #finding the first occurence of p in t using binary search
        
        #top and bottom of the interval under current scrutiny
        left, right = 0, len(self.td)

        #assuming the interval is still valid, shrink the interval
        while left < right:
            #getting the suffix at the current midpoint
            midpoint = (left + right)//2
            midpointSuffix = self.td[self.index[midpoint]:self.index[midpoint] + len(p)]
            
            #comparing the current midpoint suffix to the pattern
            if midpointSuffix < p:
                #shrinking the interval from the bottom
                #why plus 1??? to preserve while condition I believe
                left = midpoint + 1
            else:
                #shrinking the interval by setting the right end to the midpoint
                right = midpoint
        
        lowOccurence = left
        newLeft = lowOccurence
        right = len(self.td)
        
        #having found the earliest instance of the pattern, find the absolute last
        while newLeft < right:
            #getting the suffix at the current midpoint 
            midpoint = (newLeft+right)//2

            #same as before
            midpointSuffix = self.td[self.index[midpoint]:self.index[midpoint] + len(p)]
            
            #comparing the current midpoint suffix to the pattern, but searching for last occurence
            if midpointSuffix > p:
                right = midpoint
            else:
                newLeft = midpoint + 1
            
            highOcurrence = right 
            
            
        #print range of occurrences
        return self.index[lowOccurence:highOcurrence]
        
        
thisSA = SuffixArray("ACTTGGAGATCTTTGAGGCTAGGTATTCGGGATCGAAGCTCATTTCGGGGATCGATTACGATATGGTGGGTATTCGGGA")
thisSA.query('GGTATTCGGGA')

[68, 21]

In [5]:
# Test suffix array construction
sa = SuffixArray("ATA")
sa.index == [ 3, 2, 0, 1 ]

True

In [6]:
# Test suffix array search
text = 'ACTTGGAGATCTTTGAGGCTAGGTATTCGGGATCGAAGCTCATTTCGGGGATCGATTACGATATGGTGGGTATTCGGGA'
pattern = 'GGTATTCGGGA'
sa = SuffixArray(text)
sorted(sa.query(pattern)) == [21, 68]

True